In [1]:
import arcpy
import equations

In [2]:
arcpy.env.workspace = "C:/Users/rucke/SpeedHumpProject/SpeedHump.gdb"
fc = "SpeedHump/SpeedHump20"
edit = arcpy.da.Editor(arcpy.env.workspace)

## Accident History Flag

In [4]:
fields = ["TEYA", "FYA", "THYA", "AH"]

edit.startEditing(False, True)
edit.startOperation()

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        row[3] = equations.acc_hist_flag(row[0], row[1], row[2])
        print("E: ", row[3])
        cursor.updateRow(row)

edit.stopOperation()
edit.stopEditing(True)

E:  
E:  Hi 10 yr crash
E:  Hi 5 yr crash
E:  Hi 10 yr crash
E:  
E:  Hi 10 yr crash
E:  Hi 10 yr crash
E:  
E:  Hi 10 yr crash
E:  Hi 10 yr crash
E:  
E:  Hi 10 yr crash
E:  Hi 10 yr crash
E:  Hi 5 yr crash
E:  Hi 10 yr crash
E:  Hi 10 yr crash
E:  Hi 5 yr crash
E:  
E:  Hi 10 yr crash
E:  Hi 10 yr crash


## Pre Score

In [5]:
fields = ["THYA", "FYA", "DA", "SR", "BDR", "Park", "OSF", "SWI", "SL", "SW", "CDP", "PS"]

edit.startEditing(False, True)
edit.startOperation()

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        sf_int = []
        for sf in row[2:7]:
            sf_int.append(1 if sf == "Yes" else 0)

        sw = True if row[9] == "Yes" else False
        cdp = True if row[10] == "Yes" else False
        row[11] = float(equations.pre_score(row[0], row[1], sf_int, row[7], row[8], sw, cdp))
        print(row[11] - 0.5)
        cursor.updateRow(row)

edit.stopOperation()
edit.stopEditing(True)

4.0120000000000005
7.504999999999999
9.346
36.717
4.675
14.666
43.729
5.366
20.729
13.475
2.1
23.64
16.019
16.633
16.647
8.001
8.05
6.929
14.0
16.675


## Speed Calculation

In [6]:
fields = ["FN_E_S", "FS_W_S", "SN_E_S", "SS_W_S", "CL85"]

edit.startEditing(False, True)
edit.startOperation()

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        row[4] = equations.calc_speed(row[0], row[1], row[2], row[3])
        print(row[4])
        cursor.updateRow(row)

edit.stopOperation()
edit.stopEditing(True)

36.185001373291016
43.29999923706055
33.02499961853027
32.14999961853027
33.959999084472656
37.75
38.55500030517578
24.5
41.314998626708984
36.83500099182129
-1
39.2549991607666
32.5
35.53999900817871
30.199999809265137
33.5
34.970001220703125
27.399999618530273
34.170000076293945
39.619998931884766


## ADT Calculation

In [7]:
fields = ["FN_E_T", "FS_W_T", "SN_E_T", "SS_W_T", "CLADT"]

edit.startEditing(False, True)
edit.startOperation()

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        row[4] = equations.calc_adt(row[0], row[1], row[2], row[3])
        print(row[4])
        cursor.updateRow(row)

edit.stopOperation()
edit.stopEditing(True)

1132
1387
691
1986
864
1794
12391
1765
2025
1218
-1
4261
2527
1311
863
606
871
277
5740
5908


## Rank Score

In [8]:
fields = ["SP", "CL85", "CLADT", "KSI", "THYA" "FYA", "SW", "CDP", "DA", "SR", "BDR", "Park", "OSF", "SWI", "RS"]

edit.startEditing(False, True)
edit.startOperation()

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        width_factor = 1 if row[13] < 36 else 0
        sw_factor = 0.5 if row[6] == "No" else 0
        cd_factor = 0.5 if row[7] == "Yes" else 0
        
        sf_int = []
        for sf in row[9:13]:
            sf_int.append(1 if sf == "Yes" else 0)
        sf_points = equations.calc_sf_points(sf_int, width_factor)

        da = 1 if row[8] == "Yes" else 0

        row[14] = equations.rank_score(row[0], row[1], row[2], row[3], row[4], row[5], sw_factor + cd_factor, da, sf_points)
        print("Rank Score", row[14] - 5)
        print("")
        cursor.updateRow(row)

edit.stopOperation()
edit.stopEditing(True)

CK:  18.555004119873047
CL:  0.33
CM:  10
CO:  0
Rank Score 28.885004119873045

CK:  39.89999771118164
CL:  0.9675
CM:  20
CO:  0
Rank Score 60.86749771118164

CK:  9.075004577636719
CL:  0
CM:  20
CO:  0
Rank Score -5.0

CK:  6.450004577636719
CL:  2.465
CM:  20
CO:  0
Rank Score 28.915004577636722

CK:  11.879997253417969
CL:  0
CM:  10
CO:  0
Rank Score -5.0

CK:  23.25
CL:  1.985
CM:  20
CO:  0
Rank Score 45.235

CK:  10.665000915527344
CL:  20
CM:  20
CO:  0
Rank Score 50.665000915527344

CK:  0
CL:  1.9125
CM:  10
CO:  26
Rank Score -5.0

CK:  33.94499588012695
CL:  2.5625
CM:  20
CO:  0
Rank Score 56.50749588012695

CK:  20.50499725341797
CL:  0.545
CM:  20
CO:  0
Rank Score 41.04999725341797

CK:  0
CL:  0
CM:  0
CO:  0
Rank Score -5.0

CK:  27.764991760253906
CL:  8.1525
CM:  20
CO:  26
Rank Score 81.91749176025391

CK:  7.5
CL:  3.8175
CM:  20
CO:  26
Rank Score 57.317499999999995

CK:  16.62000274658203
CL:  0.7775
CM:  20
CO:  30
Rank Score 67.39750274658203

CK:  0.6000022